In [7]:
from pathlib import Path

def getAllOldCsv():
    output = []
    files = list(Path(
        "/Users/harshitagrawal/Desktop/projects/financial_freedom_projects/options_backtesting_bot/data/weekly_data/").rglob("*.[cC][sS][vV]"))
    for file in files:

        filepath = ("/".join(file.parts)
                    ).replace("//", "/")
        output.append(filepath)
    return output


oldFiles = getAllOldCsv()
print(oldFiles[0:5])

['/Users/harshitagrawal/Desktop/projects/financial_freedom_projects/options_backtesting_bot/data/weekly_data/2022-09-02_2022-09-08.csv', '/Users/harshitagrawal/Desktop/projects/financial_freedom_projects/options_backtesting_bot/data/weekly_data/2021-09-03_2021-09-09.csv', '/Users/harshitagrawal/Desktop/projects/financial_freedom_projects/options_backtesting_bot/data/weekly_data/2021-11-12_2021-11-18.csv', '/Users/harshitagrawal/Desktop/projects/financial_freedom_projects/options_backtesting_bot/data/weekly_data/2022-01-21_2022-01-27.csv', '/Users/harshitagrawal/Desktop/projects/financial_freedom_projects/options_backtesting_bot/data/weekly_data/2021-05-28_2021-06-03.csv']


In [14]:
import csv
from datetime import datetime, timedelta
from helper import generateFiles, writeFiles
from candleStick import CandleStickDataType, TimeFrameTypeEnum, CandleStickListDataType, InstrumentTypeEnum

def readOptionsCsvFile(filename: str):
        output = []
        maxTimestamp = 0

        with open(filename, mode='r')as file:
            csvFile = csv.reader(file)
            for lines in csvFile:
                if lines[0].strip().lower() == '':
                    continue
                if lines[0].strip().lower() == 'ticker':
                    continue

                date = lines[1].strip().split(" ")[0]
                time = lines[1].strip().split(" ")[1]

                dateEntries = date.split("-")
                timeEntries = time.split(":")

                timestamp = datetime(
                    int(dateEntries[2]),
                    int(dateEntries[1]),
                    int(dateEntries[0]),
                    int(timeEntries[0]),
                    int(timeEntries[1]),
                    int(timeEntries[2]) if len(timeEntries) == 3 else 0
                ) + timedelta(seconds=19800)

                if int(timestamp.timestamp()) > maxTimestamp:
                    maxTimestamp = int(timestamp.timestamp())

                output.append(
                    {
                        "ticker": lines[0].lower().strip().upper() if "BANKNIFTY" not in lines[0].lower().strip().upper() else lines[0].lower().strip().upper().replace("BANKNIFTY", "CNXBAN"),
                        "timestamp": int(timestamp.timestamp()),
                        "open": float(lines[2].lower().strip()),
                        "high": float(lines[3].lower().strip()),
                        "low": float(lines[4].lower().strip()),
                        "close": float(lines[5].lower().strip()),
                        "volume": int(lines[6].lower().strip()),
                        "open_interest": int(lines[7].lower().strip())
                    }
                )

        expiry = str(datetime.fromtimestamp(
            maxTimestamp).year) + '-' + str(datetime.fromtimestamp(
                maxTimestamp).month) + '-' + str(datetime.fromtimestamp(
                    maxTimestamp).day) + "T21:00:00.000"
        return output, expiry


def convertOptionsCsvFile(fileData: dict, expiry: str) -> CandleStickListDataType:
    try:
        output = CandleStickListDataType(data=[])
        for entry in fileData:
            if "WK" not in entry["ticker"].upper():
                continue

            if "MIDCAP" in entry["ticker"].upper():
                continue

            data = entry["ticker"].upper().split("WK")

            ticker = data[0]

            if ticker != "CANBAN" and ticker != "NIFTY":
                continue

            strike = int(data[1][0:-2])
            
            contract = ""
            if str(data[1]).replace(data[1][0:-2], "") == "CE":
                contract = "call"
            elif str(data[1]).replace(data[1][0:-2], "") == "PE":
                contract = "put"
            
            if contract == "":
                continue

            output.data.append(
                CandleStickDataType(
                    ticker=ticker,
                    open=entry["open"],
                    high=entry["high"],
                    low=entry["low"],
                    close=entry["close"],
                    timestamp=entry["timestamp"],
                    iso_string=datetime.fromtimestamp(entry["timestamp"]).isoformat(),
                    volume=entry["volume"],
                    open_interest=entry["open_interest"],
                    strike=strike,
                    expiry_type="weekly",
                    expiry=expiry,
                    instrument=InstrumentTypeEnum.OPTIONS,
                    contract_type=contract,
                    timeframe=TimeFrameTypeEnum.MINUTE_1
                )
            )

        return output
    except Exception as e:
        print(e)
        

def convertCashCsvFile(fileData: dict) -> CandleStickListDataType:
    try:
        output = CandleStickListDataType(data=[])
        for entry in fileData:
            if entry["ticker"].upper().strip() != "NIFTY" and entry["ticker"].upper().strip() != "CANBAN":
                continue


            ticker = entry["ticker"]
            

            output.data.append(
                CandleStickDataType(
                    ticker=ticker,
                    open=entry["open"],
                    high=entry["high"],
                    low=entry["low"],
                    close=entry["close"],
                    timestamp=entry["timestamp"],
                    iso_string=datetime.fromtimestamp(entry["timestamp"]).isoformat(),
                    volume=entry["volume"],
                    instrument=InstrumentTypeEnum.CASH,
                    timeframe=TimeFrameTypeEnum.MINUTE_1
                )
            )

        return output
    except Exception as e:
        print(e)
        

file = "/Users/harshitagrawal/Desktop/projects/financial_freedom_projects/options_backtesting_bot/data/weekly_data/2021-08-20_2021-08-26.csv"
print(f'Processing -> {file.split("/")[len(file.split("/")) - 1]}')
fileData, expiryDate = readOptionsCsvFile(file)
data = convertOptionsCsvFile(fileData, expiryDate)
data = generateFiles(data, InstrumentTypeEnum.OPTIONS)
writeFiles(data, InstrumentTypeEnum.OPTIONS)


Processing -> 2021-08-20_2021-08-26.csv


True

In [ ]:
import csv
from datetime import datetime, timedelta
from helper import generateFiles, writeFiles
from candleStick import CandleStickDataType, TimeFrameTypeEnum, CandleStickListDataType, InstrumentTypeEnum
from pathlib import Path

oldFiles = getAllOldCsv()

for file in oldFiles:

    print(f'Processing -> {file.split("/")[len(file.split("/")) - 1]}')

    fileData, expiryDate = readOptionsCsvFile(file)
    data = convertCashCsvFile(fileData)
    data = generateFiles(data, InstrumentTypeEnum.CASH)
    writeFiles(data, InstrumentTypeEnum.CASH)


In [15]:
import csv
from datetime import datetime, timedelta
from helper import generateFiles, writeFiles
from candleStick import CandleStickDataType, TimeFrameTypeEnum, CandleStickListDataType, InstrumentTypeEnum
from pathlib import Path

oldFiles = getAllOldCsv()

for file in oldFiles:

    print(f'Processing -> {file.split("/")[len(file.split("/")) - 1]}')

    fileData, expiryDate = readOptionsCsvFile(file)
    data = convertOptionsCsvFile(fileData, expiryDate)
    data = generateFiles(data, InstrumentTypeEnum.OPTIONS)
    writeFiles(data, InstrumentTypeEnum.OPTIONS)


Processing -> 2022-09-02_2022-09-08.csv
Processing -> 2021-09-03_2021-09-09.csv
Processing -> 2021-11-12_2021-11-18.csv
Processing -> 2022-01-21_2022-01-27.csv
Processing -> 2021-05-28_2021-06-03.csv
Processing -> 2022-03-04_2022-03-10.csv
Processing -> 2021-08-20_2021-08-26.csv
Processing -> 2021-02-12_2021-02-18.csv
Processing -> 2021-03-05_2021-03-10.csv
Processing -> 2022-06-03_2022-06-09.csv
Processing -> 2021-09-17_2021-09-23.csv
Processing -> 2021-10-29_2021-11-03.csv
Processing -> 2022-09-16_2022-09-22.csv
Processing -> 2021-01-01_2021-01-07.csv


Processing -> 2021-07-09_2021-07-15.csv
Processing -> 2022-07-08_2022-07-14.csv
Processing -> 2022-03-25_2022-03-31.csv
Processing -> 2021-07-30_2021-08-05.csv
Processing -> 2022-05-27_2022-06-02.csv
Processing -> 2022-09-23_2022-09-29.csv
Processing -> 2021-01-08_2021-01-14.csv
Processing -> 2022-06-17_2022-06-23.csv
Processing -> 2022-03-11_2022-03-17.csv
Processing -> 2022-07-01_2022-07-07.csv
Processing -> 2022-01-07_2022-01-13.csv
Processing -> 2021-06-18_2021-06-24.csv
Processing -> 2022-07-29_2022-08-04.csv
Processing -> 2021-11-19_2021-11-25.csv
Processing -> 2021-10-01_2021-10-07.csv
Processing -> 2021-12-24_2021-12-30.csv
Processing -> 2022-04-18_2022-04-21.csv
Processing -> 2021-04-16_2021-04-22.csv
Processing -> 2022-08-05_2022-08-11.csv
Processing -> 2022-02-18_2022-02-24.csv
Processing -> 2021-02-19_2021-02-25.csv


Processing -> 2021-01-29_2021-02-04.csv
Processing -> 2021-10-08_2021-10-14.csv
Processing -> 2022-04-22_2022-04-28.csv
Processing -> 2021-04-23_2021-04-29.csv
Processing -> 2021-12-10_2021-12-16.csv
Processing -> 2021-05-07_2021-05-12.csv
Processing -> 2022-10-07_2022-10-13.csv
Processing -> 2021-08-27_2021-09-02.csv
Processing -> 2021-03-26_2021-04-01.csv
Processing -> 2022-10-21_2022-10-27.csv
Processing -> 2022-08-26_2022-09-01.csv
Processing -> 2022-03-21_2022-03-24.csv
Processing -> 2022-01-28_2022-02-03.csv
Processing -> 2021-04-05_2021-04-08.csv
Processing -> 2022-05-20_2022-05-26.csv
Processing -> 2021-05-21_2021-05-27.csv
Processing -> 2022-02-11_2022-02-17.csv
Processing -> 2021-07-16_2021-07-22.csv
Processing -> 2022-09-09_2022-09-15.csv
Processing -> 2021-12-17_2021-12-23.csv
Processing -> 2021-01-22_2021-01-28.csv
Processing -> 2022-05-13_2022-05-19.csv
Processing -> 2021-07-23_2021-07-29.csv


Processing -> 2022-07-22_2022-07-28.csv
Processing -> 2022-10-14_2022-10-20.csv
Processing -> 2021-10-15_2021-10-20.csv
Processing -> 2021-05-14_2021-05-20.csv
Processing -> 2021-03-12_2021-03-18.csv
Processing -> 2022-08-19_2022-08-25.csv
Processing -> 2021-12-03_2021-12-09.csv
Processing -> 2021-02-05_2021-02-11.csv
Processing -> 2022-02-04_2022-02-10.csv
Processing -> 2022-04-29_2022-05-05.csv
Processing -> 2021-11-05_2021-11-11.csv
Processing -> 2021-02-26_2021-03-04.csv
Processing -> 2021-09-13_2021-09-16.csv
Processing -> 2021-11-26_2021-12-02.csv
Processing -> 2021-06-04_2021-06-10.csv
Processing -> 2021-12-31_2022-01-06.csv
Processing -> 2021-04-30_2021-05-06.csv
Processing -> 2022-04-08_2022-04-13.csv
Processing -> 2022-08-12_2022-08-18.csv
Processing -> 2021-04-09_2021-04-15.csv
Processing -> 2022-02-25_2022-03-03.csv
Processing -> 2021-10-22_2021-10-28.csv
Processing -> 2021-08-13_2021-08-18.csv
Processing -> 2022-07-15_2022-07-21.csv
Processing -> 2021-06-11_2021-06-17.csv
